In [61]:
import numpy as np 
import nltk
nltk.download('punkt')
nltk.download("stopwords")
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
from nltk.corpus import stopwords

[nltk_data] Downloading package punkt to /home/darkness/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/darkness/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/darkness/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /home/darkness/nltk_data...


## Pre Processing

### Text Retrival

In [62]:
f = open("Text.txt")
data = f.readlines()[6:]
f.close()

### Tokenization

In [63]:
# Sentence Splitting
sentences = []
for i in data:
    sentences += nltk.sent_tokenize(i)
# Word Splitting
words_in_sentence = []
for i in sentences:
    words_in_sentence.append(nltk.word_tokenize(i))

In [64]:
Stop_word = stopwords.words('english')

words_nostop_in_sentence = []
for i in words_in_sentence:
    temp = [word for word in i if word not in Stop_word]
    words_nostop_in_sentence.append(temp)

### POS Tagging

In [65]:
POS = []
for i in words_nostop_in_sentence:
    POS.append(nltk.pos_tag(i))

### Lemmatization

In [72]:
root_per_sentence = []
obj = nltk.WordNetLemmatizer()

for i in words_nostop_in_sentence:
    temp  = [obj.lemmatize(j) for j in i ] 
    root_per_sentence.append(temp)

## END